[GSE217483](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE217483)

在这里，我们分析了感染严重急性呼吸系统综合征冠状病毒2型的恒河猴的肺、肝、肾和大脑皮层的约77000个单核转录组。


| tissue | status | batch | gender | macacaID | macacaID_count |
| :--: | :--: | :--: | :--: | :--: | :--: |
| cortex | healthy | 0 | male | Ctrl1 | 1996 |
| cortex | healthy | 1 | male | Ctrl2 | 8349 |
| cortex | infected | 0 | female | Case1 | 5827 |
| cortex | infected | 0 | male | Case2 | 7434 |
| kidney | healthy | 0 | male | Ctrl1 | 5307 |
| kidney | infected | 0 | female | Case1 | 4733 |
| kidney | infected | 0 | male | Case2 | 3510 |
| liver | healthy | 0 | male | Ctrl1 | 4152 |
| liver | infected | 0 | female | Case1 | 7510 |
| liver | infected | 0 | male | Case2 | 6070 |
| lung | healthy | 0 | male | Ctrl1 | 5360 |
| lung | infected | 0 | female | Case1 | 9035 |
| lung | infected | 0 | male | Case2 | 7746 |

In [ ]:
import sys
from pathlib import Path
p_root = Path('link','res_publish')
None if str(p_root) in sys.path else sys.path.append(str(p_root))


In [ ]:
from func import *

In [ ]:
adata = sc.read_h5ad('GSE217483_Macaca_global_integration.h5ad')
display(type(adata.X))
adata.X = csr_matrix(adata.X)
display(type(adata.X), adata)

In [ ]:
temp = adata.X[:100, :100]
display(np.ravel(temp[temp > 0])[:10])
del temp

In [ ]:
print(*adata.obs.columns, sep='\n')

In [ ]:
display(adata.obs.head(2))
# for i in 'sampleID,status,macacaID,batch,gender,organ'.split(','):
#     print(i.ljust(50,'-'),end='\n\n')
#     display(adata.obs[i].value_counts())
adata.obs = adata.obs.loc[:,
                          'sampleID,fine_cell_type,status,macacaID,batch,gender,organ'.split(',')]
adata.obs = adata.obs.rename(columns={'organ': 'tissue','fine_cell_type':'pre_fine_cell_type'})
adata.obs['macacaID'] = adata.obs['macacaID'].str.replace(
    'Ctrl', 'healthy_').str.replace('Case', 'infected_')
adata.obs['_batch'] = adata.obs['macacaID']
# new
adata.obs['cell_type'] = adata.obs['pre_fine_cell_type'].map(
    lambda k:{'Macrophage':'macrophage'}.setdefault(k,'')
)
adata.obs = adata.obs.loc[:,
                          '_batch,sampleID,cell_type,pre_fine_cell_type,status,macacaID,batch,gender,tissue'.split(',')]
display(adata.obs.head(2), adata.obs['pre_fine_cell_type'].value_counts())

# lung

In [ ]:
adata_l = adata[adata.obs['tissue'] == 'lung', :]
display(
    adata_l,
    adata_l.obs.head(2),
    adata_l.obs['tissue'].value_counts(),
    adata_l.obs['_batch'].value_counts(),
    adata_l.obs['pre_fine_cell_type'].value_counts())

In [ ]:
# new
adata_l_marc = adata_l[adata_l.obs['cell_type'] == 'macrophage']
display(adata_l_marc,adata_l_marc.obs.head(2),adata_l_marc.obs['cell_type'].value_counts())

In [ ]:
adata_l_marc.obs['macr_cell_type'] =adata_l_marc.obs['cell_type']
adata_l_marc.obs = adata_l_marc.obs.loc[:,
                          '_batch,sampleID,cell_type,macr_cell_type,pre_fine_cell_type,status,macacaID,batch,gender,tissue'.split(',')]
display(adata_l_marc,adata_l_marc.obs.head(2))

In [ ]:
# display(adata.obs['tissue'].value_counts())
# for tissue in adata.obs['tissue'].unique():
#     h5ad_to_mtx(adata[adata.obs['tissue'] == tissue, :],
#                 'COVID_macaca_GSE217483_{}'.format(tissue))
# print("\n[finish]\n".center(100, "-"))
h5ad_to_mtx(adata_l_marc,
p_cache.joinpath('disease','COVID_ma_macrophage_GSE217483')
           )

In [ ]:
print("\n[finish]\n".center(100,"-"))